<a href="https://colab.research.google.com/github/xFlodz/university/blob/main/%D0%94%D0%9F%D0%9E%20%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B%20Python%20%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F%20%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Практическая работа №4. Введение в Python. ООП рефакторинг**

---




**Обучающийся:** *[Введите ваше ФИО]*  



---

## **Цель работы:**



Провести рефакторинг ранее созданного Python-пакета в предыдущей практической работе, преобразовав его функциональность в соответствии с принципами объектно-ориентированного программирования. Обновленный пакет должен включать классы и методы для преобразования координат между декартовой и сферической системами координат, а также для работы с файлами.



## **Задачи:**



1. Анализ существующего пакета и определение необходимых изменений для перехода на ООП-парадигму.

2. Создание новой структуры пакета с использованием классов и модулей, соответствующих принципам ООП.

3. Реализация классов и методов для преобразования координат между декартовой и сферической системами координат.

4. Реализация классов и методов для работы с файлами, обеспечивая удобный интерфейс для чтения и записи данных.

5. Создание файла `__main__.py` с консольным интерфейсом для взаимодействия с функциональностью пакета.

6. Тестирование и проверка работоспособности обновленного пакета на примерах, подтверждение корректности реализации.



## **Демонстрация результата:**



Вставьте код каждого из ваших модулей в соответствующие ячейки ниже.

**Строку, начинающуюся на %%writefile... стирать запрещено**

### **Содержимое модуля \_\_init__.py:**

In [1]:
!mkdir geo_transform

In [2]:
%%writefile geo_transform/__init__.py

from .transformations import CoordinateTransformer
from .utils import AngleConverter
from .file_operations import FileManager

__all__ = [
    'CoordinateTransformer',
    'AngleConverter',
    'FileManager'
]

Writing geo_transform/__init__.py


### **Содержимое модуля transformations.py:**

In [3]:
%%writefile geo_transform/transformations.py

import math
from .utils import AngleConverter

class CoordinateTransformer:
    @staticmethod
    def cartesian_to_spherical(x, y, z, to_degrees=True):
        r = math.sqrt(x**2 + y**2 + z**2)

        theta = math.atan2(y, x)

        if r == 0:
            phi = 0
        else:
            phi = math.acos(z / r)

        if to_degrees:
            theta = AngleConverter.rad_to_deg(theta)
            phi = AngleConverter.rad_to_deg(phi)

        return r, theta, phi

    @staticmethod
    def spherical_to_cartesian(r, theta, phi, from_degrees=True):
        if from_degrees:
            theta = AngleConverter.deg_to_rad(theta)
            phi = AngleConverter.deg_to_rad(phi)

        x = r * math.sin(phi) * math.cos(theta)
        y = r * math.sin(phi) * math.sin(theta)
        z = r * math.cos(phi)

        return x, y, z

    @staticmethod
    def transform_point(point, transformation_type, **kwargs):
        if transformation_type == 'cartesian_to_spherical':
            return CoordinateTransformer.cartesian_to_spherical(*point, **kwargs)
        elif transformation_type == 'spherical_to_cartesian':
            return CoordinateTransformer.spherical_to_cartesian(*point, **kwargs)
        else:
            raise ValueError("Неизвестный тип преобразования")


Writing geo_transform/transformations.py


### **Содержимое модуля utils.py:**

In [4]:
%%writefile geo_transform/utils.py

import math

class AngleConverter:
    @staticmethod
    def deg_to_rad(degrees):
        return degrees * math.pi / 180

    @staticmethod
    def rad_to_deg(radians):
        return radians * 180 / math.pi

    @staticmethod
    def normalize_angle(angle, in_degrees=True):
        if in_degrees:
            return angle % 360
        else:
            return angle % (2 * math.pi)

Writing geo_transform/utils.py


### **Содержимое модуля file_operations.py:**

In [5]:
%%writefile geo_transform/file_operations.py

class FileManager:
    @staticmethod
    def write_results_to_file(results, filename, mode='w'):
        with open(filename, mode, encoding='utf-8') as f:
            for i, result in enumerate(results, 1):
                f.write(f"{i}. {result}\n")

    @staticmethod
    def read_coordinates_from_file(filename, coord_type='auto'):
        coordinates = []
        with open(filename, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                line = line.strip()
                if not line or line.startswith('#'):
                    continue

                try:
                    parts = line.split()
                    coords = [float(x) for x in parts]

                    if len(coords) == 3:
                        if coord_type == 'auto':
                            if coords[0] > 10 and all(c >= 0 for c in coords):
                                coord_info = (coords, 'spherical')
                            else:
                                coord_info = (coords, 'cartesian')
                        else:
                            coord_info = (coords, coord_type)

                        coordinates.append(coord_info)
                    else:
                        print(f"Предупреждение: строка {line_num} содержит {len(coords)} координат вместо 3")

                except ValueError as e:
                    print(f"Ошибка в строке {line_num}: {e}")

        return coordinates

    @staticmethod
    def batch_transform_coordinates(filename, output_file=None):
        if output_file is None:
            output_file = f"transformed_{filename}"

        coordinates = FileManager.read_coordinates_from_file(filename, 'auto')
        results = []

        for i, (coords, coord_type) in enumerate(coordinates, 1):
            try:
                if coord_type == 'cartesian':
                    x, y, z = coords
                    r, theta, phi = CoordinateTransformer.cartesian_to_spherical(x, y, z)
                    result = f"Декартовы: ({x}, {y}, {z}) -> Сферические: (r={r:.2f}, θ={theta:.2f}°, φ={phi:.2f}°)"

                elif coord_type == 'spherical':
                    r, theta, phi = coords
                    x, y, z = CoordinateTransformer.spherical_to_cartesian(r, theta, phi)
                    result = f"Сферические: (r={r}, θ={theta}°, φ={phi}°) -> Декартовы: ({x:.2f}, {y:.2f}, {z:.2f})"

                results.append(result)
                print(f"{i}. {result}")

            except Exception as e:
                error_msg = f"Ошибка преобразования координат {coords}: {e}"
                results.append(error_msg)
                print(f"{i}. {error_msg}")

        FileManager.write_results_to_file(results, output_file)
        print(f"\nРезультаты сохранены в {output_file}")

        return results

Writing geo_transform/file_operations.py


### **Содержимое модуля \_\_main__.py:**

In [6]:
%%writefile geo_transform/__main__.py

import sys
from .transformations import CoordinateTransformer
from .file_operations import FileManager

class CoordinateConverterApp:
    def __init__(self):
        self.transformer = CoordinateTransformer
        self.file_manager = FileManager

    def display_menu(self):
        """Отображает главное меню"""
        print("\n" + "="*50)
        print("    КОНВЕРТЕР КООРДИНАТ (ООП РЕФАКТОРИНГ)")
        print("="*50)
        print("1. Декартовы → Сферические")
        print("2. Сферические → Декартовы")
        print("3. Обработать файл с координатами")
        print("4. Пакетное преобразование файла")
        print("5. Выход")
        print("="*50)

    def cartesian_to_spherical_interactive(self):
        """Интерактивное преобразование декартовых в сферические"""
        print("\n--- Декартовы → Сферические ---")
        try:
            x = float(input("Введите x: "))
            y = float(input("Введите y: "))
            z = float(input("Введите z: "))

            r, theta, phi = self.transformer.cartesian_to_spherical(x, y, z)

            print(f"\nРезультат:")
            print(f"r = {r:.4f}, theta = {theta:.2f}°, phi = {phi:.2f}°")

            save = input("\nСохранить результат? (y/n): ").lower()
            if save == 'y':
                result = f"Декартовы: ({x}, {y}, {z}) -> Сферические: (r={r:.2f}, θ={theta:.2f}°, φ={phi:.2f}°)"
                self.file_manager.write_results_to_file([result], "results.txt", mode='a')
                print("Результат сохранен в results.txt")

        except ValueError:
            print("Ошибка: введите корректные числа!")
        except Exception as e:
            print(f"Ошибка преобразования: {e}")

    def spherical_to_cartesian_interactive(self):
        print("\n--- Сферические → Декартовы ---")
        try:
            r = float(input("Введите радиус r: "))
            theta = float(input("Введите азимутальный угол θ (градусы): "))
            phi = float(input("Введите полярный угол φ (градусы): "))

            x, y, z = self.transformer.spherical_to_cartesian(r, theta, phi)

            print(f"\nРезультат:")
            print(f"x = {x:.4f}, y = {y:.4f}, z = {z:.4f}")

            save = input("\nСохранить результат? (y/n): ").lower()
            if save == 'y':
                result = f"Сферические: (r={r}, θ={theta}°, φ={phi}°) -> Декартовы: ({x:.2f}, {y:.2f}, {z:.2f})"
                self.file_manager.write_results_to_file([result], "results.txt", mode='a')
                print("Результат сохранен в results.txt")

        except ValueError:
            print("Ошибка: введите корректные числа!")
        except Exception as e:
            print(f"Ошибка преобразования: {e}")

    def process_file_interactive(self):
        print("\n--- Обработка файла ---")
        filename = input("Введите имя файла: ")

        try:
            coordinates = self.file_manager.read_coordinates_from_file(filename)

            if not coordinates:
                print("Файл пуст или координаты не найдены!")
                return

            print(f"\nНайдено {len(coordinates)} наборов координат:")
            results = []

            for i, (coords, coord_type) in enumerate(coordinates, 1):
                print(f"\n{i}. {coord_type}: {coords}")

                transform_type = input("Преобразовать в (c/s - карта/сфера, Enter - пропустить): ").lower()

                if transform_type == 'c' and coord_type == 'spherical':
                    r, theta, phi = coords
                    x, y, z = self.transformer.spherical_to_cartesian(r, theta, phi)
                    result = f"Сферические: {coords} -> Декартовы: ({x:.2f}, {y:.2f}, {z:.2f})"
                    results.append(result)
                    print(f"   -> {result}")

                elif transform_type == 's' and coord_type == 'cartesian':
                    x, y, z = coords
                    r, theta, phi = self.transformer.cartesian_to_spherical(x, y, z)
                    result = f"Декартовы: {coords} -> Сферические: (r={r:.2f}, θ={theta:.2f}°, φ={phi:.2f}°)"
                    results.append(result)
                    print(f"   -> {result}")

            if results:
                save = input("\nСохранить все результаты? (y/n): ").lower()
                if save == 'y':
                    self.file_manager.write_results_to_file(results, "file_processing_results.txt")
                    print("Результаты сохранены в file_processing_results.txt")

        except FileNotFoundError:
            print(f"Файл {filename} не найден!")
        except Exception as e:
            print(f"Ошибка при обработке файла: {e}")

    def batch_transform_file(self):
        print("\n--- Пакетное преобразование файла ---")
        filename = input("Введите имя файла: ")

        try:
            self.file_manager.batch_transform_coordinates(filename)
        except Exception as e:
            print(f"Ошибка: {e}")

    def run(self):
        print("Добро пожаловать в конвертер координат (ООП версия)!")

        while True:
            self.display_menu()
            choice = input("\nВыберите действие (1-5): ").strip()

            if choice == '1':
                self.cartesian_to_spherical_interactive()
            elif choice == '2':
                self.spherical_to_cartesian_interactive()
            elif choice == '3':
                self.process_file_interactive()
            elif choice == '4':
                self.batch_transform_file()
            elif choice == '5':
                print("\nВыход из программы. До свидания!")
                break
            else:
                print("Неверный выбор! Попробуйте снова.")


def main():
    app = CoordinateConverterApp()
    app.run()


if __name__ == "__main__":
    main()

Writing geo_transform/__main__.py


### **Содержимое модуля main.py (с импортом пакета и тестированием функций из него):**

In [7]:
from geo_transform import CoordinateTransformer, AngleConverter, FileManager

print("1. Тест преобразований координат:")
x, y, z = 1, 1, 1
r, theta, phi = CoordinateTransformer.cartesian_to_spherical(x, y, z)
print(f"   Декартовы ({x}, {y}, {z}) -> Сферические: r={r:.2f}, θ={theta:.2f}°, φ={phi:.2f}°")

x_back, y_back, z_back = CoordinateTransformer.spherical_to_cartesian(r, theta, phi)
print(f"   Обратное преобразование -> Декартовы: x={x_back:.2f}, y={y_back:.2f}, z={z_back:.2f}")

print("\n2. Тест преобразования углов:")
degrees = 180
radians = AngleConverter.deg_to_rad(degrees)
print(f"   {degrees}° = {radians:.2f} радиан")
print(f"   {radians:.2f} радиан = {AngleConverter.rad_to_deg(radians):.2f}°")

print("\n3. Тест работы с файлами:")
test_results = [
    "Тестовая строка 1",
    "Тестовая строка 2",
    "Тестовая строка 3"
]
FileManager.write_results_to_file(test_results, "test_output.txt")
print("   Файл test_output.txt создан")

print("\n4. Тест универсального метода:")
point = (1, 1, 1)
result = CoordinateTransformer.transform_point(point, 'cartesian_to_spherical')
print(f"   Универсальное преобразование {point} -> {result}")

1. Тест преобразований координат:
   Декартовы (1, 1, 1) -> Сферические: r=1.73, θ=45.00°, φ=54.74°
   Обратное преобразование -> Декартовы: x=1.00, y=1.00, z=1.00

2. Тест преобразования углов:
   180° = 3.14 радиан
   3.14 радиан = 180.00°

3. Тест работы с файлами:
   Файл test_output.txt создан

4. Тест универсального метода:
   Универсальное преобразование (1, 1, 1) -> (1.7320508075688772, 45.0, 54.735610317245346)
